In [17]:
import mysql.connector
from datetime import datetime, timedelta
import json
import random

EVENT_TYPE = []


def get_conn():
    try:
        conn = mysql.connector.connect(
            host="db-project.clt5dhnemxsz.us-east-2.rds.amazonaws.com",  # No 'https://' prefix
            user="admin",
            password="12345678",
            database="shems",
            port=3306,  # or your custom port
        )
        return conn

    # Database operations go here

    except mysql.connector.Error as e:
        print(f"Error connecting to MySQL: {e}")
        return None


def close_conn(conn):
    if conn.is_connected:
        conn.close()

def get_Refrigerator(conn):
    cursor = conn.cursor()
    cursor.execute(
        'SELECT device_id, model_name FROM device_model NATURAL JOIN device_registered WHERE model_type = "Refrigerator"'
    )
    refrigerator_ids = cursor.fetchall()
    LG_rf = []
    Samsung_rf = []
    for item in refrigerator_ids:
        id, brand = item
        if "LG" in brand:
            LG_rf.append(id)
        else:
            Samsung_rf.append(id)
    return Samsung_rf, LG_rf

connection = get_conn()
Samsung_rf, LG_rf = get_Refrigerator(connection)
Samsung_rf.sort()
LG_rf.sort()
print(Samsung_rf)
print(LG_rf)

[4, 15, 26, 37]
[10, 21, 32]


In [24]:
class Refrigerator:
    def __init__(self,id:int,brand,start_date:datetime,end_date:datetime):
        self.id = id
        self.FridgeEnergyPerHour = 0.025 if brand == "LG" else 0.03
        self.start_date = start_date
        self.end_date = end_date
    def genData(self):
        data = []
        current_date = self.start_date

        while current_date <= self.end_date:
            if current_date.day % 5 == 0:
                open_hour = random.randint(0, 7)
                open_duration = random.randint(1, 32) * 5  # Random duration in minutes

                # Fridge is turned on at the start of the day
                on_time = datetime(current_date.year, current_date.month, current_date.day, 0, 0)
                data.append((self.id, on_time, 'On', None))

                # Fridge door open event
                open_time = datetime(current_date.year, current_date.month, current_date.day, open_hour, 0)

                # Energy reports and door close event
                minute = 0
                while minute < 1440:  # Loop through the day
                    time = current_date + timedelta(minutes=minute)


                    energy = round(self.FridgeEnergyPerHour * 1.5,3) if open_time <= time < open_time + timedelta(minutes=open_duration) else self.FridgeEnergyPerHour
                    data.append((self.id, time, 'EnergyReport', energy))

                    # Fridge door close event
                    if time == open_time:
                        data.append((self.id, open_time, 'DoorOpen', None))
                    if time == open_time + timedelta(minutes=open_duration):
                        data.append((self.id, time, 'DoorClose', None))


                    minute += 5  # Increase by 5 minutes

                # Fridge is turned off at the end of the day
                off_time = datetime(current_date.year, current_date.month, current_date.day, 23, 59)
                data.append((self.id, off_time, 'Off', None))

            current_date += timedelta(days=1)

        return data
start_date = datetime(2022, 8, 1)
end_date = datetime(2022, 9, 30)
ref1 = Refrigerator(1,"LG",start_date, end_date)
ref1.genData()

[(1, datetime.datetime(2022, 8, 5, 0, 0), 'On', None),
 (1, datetime.datetime(2022, 8, 5, 0, 0), 'EnergyReport', 0.025),
 (1, datetime.datetime(2022, 8, 5, 0, 5), 'EnergyReport', 0.025),
 (1, datetime.datetime(2022, 8, 5, 0, 10), 'EnergyReport', 0.025),
 (1, datetime.datetime(2022, 8, 5, 0, 15), 'EnergyReport', 0.025),
 (1, datetime.datetime(2022, 8, 5, 0, 20), 'EnergyReport', 0.025),
 (1, datetime.datetime(2022, 8, 5, 0, 25), 'EnergyReport', 0.025),
 (1, datetime.datetime(2022, 8, 5, 0, 30), 'EnergyReport', 0.025),
 (1, datetime.datetime(2022, 8, 5, 0, 35), 'EnergyReport', 0.025),
 (1, datetime.datetime(2022, 8, 5, 0, 40), 'EnergyReport', 0.025),
 (1, datetime.datetime(2022, 8, 5, 0, 45), 'EnergyReport', 0.025),
 (1, datetime.datetime(2022, 8, 5, 0, 50), 'EnergyReport', 0.025),
 (1, datetime.datetime(2022, 8, 5, 0, 55), 'EnergyReport', 0.025),
 (1, datetime.datetime(2022, 8, 5, 1, 0), 'EnergyReport', 0.025),
 (1, datetime.datetime(2022, 8, 5, 1, 5), 'EnergyReport', 0.025),
 (1, dateti

In [27]:
start_date = datetime(2022, 8, 1)
end_date = datetime(2022, 9, 30)
finalData = []
for ID in LG_rf:
    ref = Refrigerator(ID,'LG',start_date,end_date)
    finalData += ref.genData()
for ID in Samsung_rf:
    ref = Refrigerator(ID,'Samsung',start_date,end_date)
    finalData += ref.genData()
finalData

[(10, datetime.datetime(2022, 8, 5, 0, 0), 'On', None),
 (10, datetime.datetime(2022, 8, 5, 0, 0), 'EnergyReport', 0.025),
 (10, datetime.datetime(2022, 8, 5, 0, 5), 'EnergyReport', 0.025),
 (10, datetime.datetime(2022, 8, 5, 0, 10), 'EnergyReport', 0.025),
 (10, datetime.datetime(2022, 8, 5, 0, 15), 'EnergyReport', 0.025),
 (10, datetime.datetime(2022, 8, 5, 0, 20), 'EnergyReport', 0.025),
 (10, datetime.datetime(2022, 8, 5, 0, 25), 'EnergyReport', 0.025),
 (10, datetime.datetime(2022, 8, 5, 0, 30), 'EnergyReport', 0.025),
 (10, datetime.datetime(2022, 8, 5, 0, 35), 'EnergyReport', 0.025),
 (10, datetime.datetime(2022, 8, 5, 0, 40), 'EnergyReport', 0.025),
 (10, datetime.datetime(2022, 8, 5, 0, 45), 'EnergyReport', 0.025),
 (10, datetime.datetime(2022, 8, 5, 0, 50), 'EnergyReport', 0.025),
 (10, datetime.datetime(2022, 8, 5, 0, 55), 'EnergyReport', 0.025),
 (10, datetime.datetime(2022, 8, 5, 1, 0), 'EnergyReport', 0.025),
 (10, datetime.datetime(2022, 8, 5, 1, 5), 'EnergyReport', 0.02

In [32]:
fp = open("genRefEvent.sql", "w")
fp.write("INSERT INTO device_event \n(device_id, event_datetime, event_label, event_number) VALUES \n")

# 使用不同的SQL字符串格式，根据是否为最后一条数据
sql = "({}, '{}', '{}', {}){}\n"

for i, data in enumerate(finalData):
    id, time, label, number = data
    # 检查是否为最后一条数据
    end_char = ';' if i == len(finalData) - 1 else ','
    fp.write(sql.format(
        id,
        time,
        label,
        "NULL" if number is None else number,
        end_char  # 使用正确的结束字符
    ))

fp.close()

In [33]:
# execute the sql
conn = get_conn()
if conn is not None:
    try:
        cursor = conn.cursor()
        with open('genRefEvent.sql', 'r') as file:
            sql_command = file.read()
        cursor.execute(sql_command)
        conn.commit()
    except mysql.connector.Error as e:
        print(f"Error executing SQL command: {e}")
    finally:
        cursor.close()
        conn.close()
else:
    print("Unable to establish database connection.")